In [21]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../input/train_folds.csv")
df.head(3)

,id,target,file_path,fold
0,00000e74ad,1,input/train/0/0/0/00000e74ad.npy,2
1,00001f4945,0,input/train/0/0/0/00001f4945.npy,1
2,0000661522,0,input/train/0/0/0/0000661522.npy,3


In [3]:
import torch

In [4]:
n_folds = 1

In [5]:
states_b0 = [torch.load(f"../final_models/v_4/b0_3d_v4/tf_efficientnet_b0_ns_fold{fold}_best_score.pth") for fold in range(n_folds)]

In [6]:
# states_b1 = [torch.load(f"../final_models/v_4/b1_3d_v4/tf_efficientnet_b1_ns_fold{fold}_best_score.pth") for fold in range(n_folds)]

In [7]:
states_b2 = [torch.load(f"../final_models/v_4/b2_3d_v4/tf_efficientnet_b2_ns_fold{fold}_best_score.pth") for fold in range(n_folds)]

In [8]:
states_b7 = [torch.load(f"../final_models/v_4/b7_3d_v4/tf_efficientnet_b7_ns_fold{fold}_best_score.pth") for fold in range(n_folds)]

In [9]:
oof_df = pd.DataFrame()
for fold in range(n_folds):
    _df = df[df['fold'] == fold].reset_index(drop=True)
    preds_b0 = states_b0[fold]['preds'].flatten()
    # preds_b1 = states_b1[fold]['preds'].flatten()
    preds_b2 = states_b2[fold]['preds'].flatten()
    preds_b7 = states_b7[fold]['preds'].flatten()
    
    _df['preds_b0'] = preds_b0
    # _df['preds_b1'] = preds_b1
    _df['preds_b2'] = preds_b2
    _df['preds_b7'] = preds_b7
    oof_df = pd.concat([oof_df, _df])

In [10]:
oof_df.head(10)

,id,target,file_path,fold,preds_b0,preds_b2,preds_b7
0,00010beb4a,1,input/train/0/0/0/00010beb4a.npy,0,0.384521,0.375488,0.392578
1,00014b7a9d,1,input/train/0/0/0/00014b7a9d.npy,0,0.213379,0.219849,0.192871
2,0001c76f56,1,input/train/0/0/0/0001c76f56.npy,0,0.650391,0.696777,0.570801
3,0002b883f5,0,input/train/0/0/0/0002b883f5.npy,0,0.186523,0.209351,0.188477
4,00034d8429,0,input/train/0/0/0/00034d8429.npy,0,0.169312,0.148560,0.113953
5,000367454d,1,input/train/0/0/0/000367454d.npy,0,0.455811,0.347656,0.371338
6,00054fd779,1,input/train/0/0/0/00054fd779.npy,0,0.918457,0.935059,0.966309
7,0006166539,1,input/train/0/0/0/0006166539.npy,0,1.000000,1.000000,1.000000
8,00061fc157,0,input/train/0/0/0/00061fc157.npy,0,0.556641,0.465820,0.429443
9,000698406f,0,input/train/0/0/0/000698406f.npy,0,0.216797,0.176392,0.183472


In [11]:
from sklearn.linear_model import LinearRegression

In [12]:
X = oof_df[['preds_b0', 'preds_b2', 'preds_b7']]
y = oof_df.target.values

In [26]:
X.values.shape

(112000, 3)

In [13]:
reg = LinearRegression().fit(X, y)

In [14]:
reg.coef_

array([0.17825375, 0.31961852, 0.48604485], dtype=float32)

In [15]:
reg.intercept_

0.016541123

In [18]:
df_test_b0 = pd.read_csv(f"../final_models/v_4/b0_3d_v4/submission.csv")
# df_test_b1 = pd.read_csv(f"../final_models/v_4/b1_3d_v4/submission.csv")
df_test_b2 = pd.read_csv(f"../final_models/v_4/b2_3d_v4/submission.csv")
df_test_b7 = pd.read_csv(f"../final_models/v_4/b7_3d_v4/submission.csv")

In [28]:
test_X = np.vstack([df_test_b0.target.values, df_test_b2.target.values, df_test_b7.target.values]).T

In [30]:
sub = pd.read_csv("../input/sample_submission.csv")

In [31]:
sub['target'] = reg.predict(test_X)

In [32]:
sub

,id,target
0,00005bced6,1.000458
1,0000806717,0.970593
2,0000ef4fe1,0.323402
3,00020de251,0.900390
4,00024887b5,0.152023
...,...,...
225995,ffff4125f1,0.284894
225996,ffff9d32a6,0.330868
225997,ffff9f4c1f,0.253899
225998,ffffa19693,1.000458


In [35]:
sub.to_csv("../final_models/v_4/submission.csv", index=False)